# Data Formats for Panel Data Analysis

There are two primary methods to express data:

  * MultiIndex DataFrames where the outer index is the entity and the inner is the time index.  This requires using pandas.
  * 3D structures were dimension 0 (outer) is variable, dimension 1 is time index and dimension 2 is the entity index.  It is also possible to use a 2D data structure with dimensions (t, n) which is treated as a 3D data structure having dimesions (1, t, n). These 3D data structures can be pandas, NumPy or xarray.

## Multi Index DataFrames
The most precise data format to use is a MultiIndex `DataFrame`.  This is the most precise since only single columns can preserve all types within a panel.  For example, it is not possible to span a single Categorical variable across multiple columns when using a pandas `Panel`. 

This example uses the jobtraining example to format a MultiIndex `DataFrame` using the `set_index` command. The entity index is known as fcode and the time index is year.

In [1]:
from linearmodels.datasets import jobtraining
data = jobtraining.load()
print(data.head())

   year   fcode  employ       sales   avgsal  scrap  rework  tothrs  union  \
0  1987  410032   100.0  47000000.0  35000.0    NaN     NaN    12.0      0   
1  1988  410032   131.0  43000000.0  37000.0    NaN     NaN     8.0      0   
2  1989  410032   123.0  49000000.0  39000.0    NaN     NaN     8.0      0   
3  1987  410440    12.0   1560000.0  10500.0    NaN     NaN    12.0      0   
4  1988  410440    13.0   1970000.0  11000.0    NaN     NaN    12.0      0   

   grant    ...     grant_1  clscrap  cgrant  clemploy   clsales    lavgsal  \
0      0    ...           0      NaN       0       NaN       NaN  10.463100   
1      0    ...           0      NaN       0  0.270027 -0.088949  10.518670   
2      0    ...           0      NaN       0 -0.063013  0.130621  10.571320   
3      0    ...           0      NaN       0       NaN       NaN   9.259130   
4      0    ...           0      NaN       0  0.080043  0.233347   9.305651   

   clavgsal  cgrant_1   chrsemp  clhrsemp  
0       NaN 

Here `set_index` is used to set the multi index using the firm code (entity) and year (time).

In [2]:
mi_data = data.set_index(['fcode', 'year'])
print(mi_data.head())

             employ       sales   avgsal  scrap  rework  tothrs  union  grant  \
fcode  year                                                                     
410032 1987   100.0  47000000.0  35000.0    NaN     NaN    12.0      0      0   
       1988   131.0  43000000.0  37000.0    NaN     NaN     8.0      0      0   
       1989   123.0  49000000.0  39000.0    NaN     NaN     8.0      0      0   
410440 1987    12.0   1560000.0  10500.0    NaN     NaN    12.0      0      0   
       1988    13.0   1970000.0  11000.0    NaN     NaN    12.0      0      0   

             d89  d88    ...     grant_1  clscrap  cgrant  clemploy   clsales  \
fcode  year              ...                                                    
410032 1987    0    0    ...           0      NaN       0       NaN       NaN   
       1988    0    1    ...           0      NaN       0  0.270027 -0.088949   
       1989    1    0    ...           0      NaN       0 -0.063013  0.130621   
410440 1987    0    0    ..

The `MultiIndex` `DataFrame` can be used to initialized the model.  When only referencing a single series, the `MultiIndex` `Series` representation can be used.

In [3]:
from linearmodels import PanelOLS
mod = PanelOLS(mi_data.lscrap, mi_data.hrsemp, entity_effect=True)
print(mod.fit())

                          PanelOLS Estimation Summary                           
Dep. Variable:                 lscrap   R-squared:                        0.0528
Estimator:                   PanelOLS   R-squared (Between):             -0.0379
No. Observations:                 140   R-squared (Within):               0.0528
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0288
Time:                        15:16:27                                           
Cov. Estimator:            Unadjusted   F-statistic:                      5.0751
                                        P-value                           0.0267
Entities:                          48   Distribution:                    F(1,91)
Avg Obs:                       2.9167                                           
Min Obs:                       1.0000   F-statistic (robust):             7.8079
Max Obs:                       3.0000   P-value                           0.0052
                            

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## pandas Panels and DataFrames
An alternative formal is to use pandas Panels ad DataFrames.  Panels should be formatted with `items` containing distinct variables, `major_axis` holding the time dimension and `minor_axis` holding the entity index.  Here we transform the MultiIndex DataFrame to a panel to demonstrate this format.

A single index DataFrame can also be used and is treated as being a single item slice of a Panel, and so the items should contain the time series dimension and the columns should contain entities. Note that using the `DataFrame` version loses information about variable names, which is not usually desirable.

In [4]:
panel = mi_data[['lscrap','hrsemp']].to_panel().swapaxes(1,2)
lscrap = panel['lscrap']
hrsemp = panel['hrsemp']
panel

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 3 (major_axis) x 157 (minor_axis)
Items axis: lscrap to hrsemp
Major_axis axis: 1987 to 1989
Minor_axis axis: 410032 to 419486

When using panels, it is best to input the panel which requires selecting using `[[`_var_`]]` so ensure that the variable(s) selected still has 3 dimensions.  This retains information about variable name.

In [5]:
res = PanelOLS(panel[['lscrap']],panel[['hrsemp']],entity_effect=True).fit()
print(res)

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 lscrap   R-squared:                        0.0528
Estimator:                   PanelOLS   R-squared (Between):             -0.0379
No. Observations:                 140   R-squared (Within):               0.0528
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0288
Time:                        15:16:27                                           
Cov. Estimator:            Unadjusted   F-statistic:                      5.0751
                                        P-value                           0.0267
Entities:                          48   Distribution:                    F(1,91)
Avg Obs:                       2.9167                                           
Min Obs:                       1.0000   F-statistic (robust):             7.8079
Max Obs:                       3.0000   P-value                           0.0052
                            

Using DataFrames removes this information and so the generic _Dep_ and _Exog_ are used.

In [6]:
res = PanelOLS(lscrap,hrsemp,entity_effect=True).fit()
print(res)

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:                    Dep   R-squared:                        0.0528
Estimator:                   PanelOLS   R-squared (Between):             -0.0379
No. Observations:                 140   R-squared (Within):               0.0528
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0288
Time:                        15:16:27                                           
Cov. Estimator:            Unadjusted   F-statistic:                      5.0751
                                        P-value                           0.0267
Entities:                          48   Distribution:                    F(1,91)
Avg Obs:                       2.9167                                           
Min Obs:                       1.0000   F-statistic (robust):             7.8079
Max Obs:                       3.0000   P-value                           0.0052
                            

## NumPy arrays
NumPy arrays are treated identically to pandas Panel and single index DataFrames.  In particular, using `panel.values` and `df.values` will produce identical results.  The main difference between NumPy and pandas is that NumPy loses all lable information.

In [7]:
res = PanelOLS(lscrap.values, hrsemp.values,entity_effect=True).fit()
print(res)

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:                    Dep   R-squared:                        0.0528
Estimator:                   PanelOLS   R-squared (Between):             -0.0379
No. Observations:                 140   R-squared (Within):               0.0528
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0288
Time:                        15:16:27                                           
Cov. Estimator:            Unadjusted   F-statistic:                      5.0751
                                        P-value                           0.0267
Entities:                          48   Distribution:                    F(1,91)
Avg Obs:                       2.9167                                           
Min Obs:                       1.0000   F-statistic (robust):             7.8079
Max Obs:                       3.0000   P-value                           0.0052
                            

## xarray DataArrays

xarray is a relatively new entrant into the set of packages used for data structures.  It is important in the context of panel models since pandas Panel is scheduled for removal in the futures, and so the only 3d data format that will remain vaiable is an xarray DataArray. `DataArray`s are similar to pandas `Panel` although they use some of their own notation and concentions.  In principle it is possible to express the same information in a `DataArray` as one can in a `Panel`

In [8]:
da = panel.to_xarray()
da

<xarray.DataArray (items: 2, major_axis: 3, minor_axis: 157)>
array([[[       nan,        nan, ...,   2.995732,        nan],
        [       nan,        nan, ...,   3.218876,        nan],
        [       nan,        nan, ...,   3.401197,        nan]],

       [[ 12.      ,  12.      , ...,   0.      ,   0.      ],
        [  3.053435,  12.      , ...,   0.      ,   0.      ],
        [  3.252033,  10.      , ...,   3.100775,  36.      ]]])
Coordinates:
  * items       (items) object 'lscrap' 'hrsemp'
  * major_axis  (major_axis) int64 1987 1988 1989
  * minor_axis  (minor_axis) int64 410032 410440 410495 410500 410501 410509 ...

In [9]:
res = PanelOLS(da.loc[['lscrap']], da.loc[['hrsemp']], entity_effect=True).fit()
print(res)

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 lscrap   R-squared:                        0.0528
Estimator:                   PanelOLS   R-squared (Between):             -0.0379
No. Observations:                 140   R-squared (Within):               0.0528
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0288
Time:                        15:16:27                                           
Cov. Estimator:            Unadjusted   F-statistic:                      5.0751
                                        P-value                           0.0267
Entities:                          48   Distribution:                    F(1,91)
Avg Obs:                       2.9167                                           
Min Obs:                       1.0000   F-statistic (robust):             7.8079
Max Obs:                       3.0000   P-value                           0.0052
                            

## Conversion of Categorical and Strings to Dummies
Categorical or string variables are treated as factors and so are converted to dummies. The first category is always dropped.  If this is not desirable, you should manually convert the data to dummies before estimating a model.

In [15]:
import pandas as pd
year_str = mi_data.reset_index()[['time']].astype('str')
year_cat = pd.Categorical(year_str.iloc[:,0])
year_str.index = mi_data.index
year_cat.index = mi_data.index
mi_data['year_str'] = year_str
mi_data['year_cat'] = year_cat

Here year has been converted to a string which is then used in the model to produce year dummies.

In [14]:
print('Exogenous variables')
print(mi_data[['hrsemp','year_str']].head())
print(mi_data[['hrsemp','year_str']].dtypes)

res = PanelOLS(mi_data[['lscrap']],mi_data[['hrsemp','year_str']],entity_effect=True).fit()
print(res)

Exogenous variables
                hrsemp year_str
entity time                    
410032 1987  12.000000     1987
       1988   3.053435     1988
       1989   3.252033     1989
410440 1987  12.000000     1987
       1988  12.000000     1988
hrsemp      float64
year_str     object
dtype: object
                          PanelOLS Estimation Summary                           
Dep. Variable:                 lscrap   R-squared:                        0.1985
Estimator:                   PanelOLS   R-squared (Between):             -0.1240
No. Observations:                 140   R-squared (Within):               0.1985
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0934
Time:                        15:22:15                                           
Cov. Estimator:            Unadjusted   F-statistic:                      7.3496
                                        P-value                           0.0002
Entities:                          48   Distribution: 

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Using categoricals has the same effect.

In [13]:
print('Exogenous variables')
print(mi_data[['hrsemp','year_cat']].head())
print(mi_data[['hrsemp','year_cat']].dtypes)

res = PanelOLS(mi_data[['lscrap']],mi_data[['hrsemp','year_cat']],entity_effect=True).fit()
print(res)

Exogenous variables
                hrsemp year_cat
entity time                    
410032 1987  12.000000     1987
       1988   3.053435     1988
       1989   3.252033     1989
410440 1987  12.000000     1987
       1988  12.000000     1988
hrsemp       float64
year_cat    category
dtype: object
                          PanelOLS Estimation Summary                           
Dep. Variable:                 lscrap   R-squared:                        0.1985
Estimator:                   PanelOLS   R-squared (Between):             -0.1240
No. Observations:                 140   R-squared (Within):               0.1985
Date:                Sun, Apr 09 2017   R-squared (Overall):             -0.0934
Time:                        15:22:05                                           
Cov. Estimator:            Unadjusted   F-statistic:                      7.3496
                                        P-value                           0.0002
Entities:                          48   Distribution

c:\git\linearmodels\linearmodels\panel\model.py:192: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
